# Functional Closures and Decorators

##  Closures
* In order to understand closures, let's review the Python scoping rules: LEGB
  * L = local
  * E = enclosing
  * G = global
  * B = builtin (e.g., len() function)
  
        a = 'global scope'
   
       def outer_func():
           b = 'local to outer_func()'
            def inner_func():
                c = 'local to inner_func()'
                print(b, 'enclosing scope')
                print(a, 'global scope')
                
* When a function references a name that is not local, Python first attempts to resolve that name in the enclosing scope
* A *closure* is a nested function which remembers a value or values from the enclosing lexical scope even when the program flow is no longer in the enclosing scope

In [1]:
def make_adder(x):
    print('id(x): %X' % id(x))
    
    def adder(y):
        return x + y # Python uses LEGB to find 'x'
    
    print('id(adder): %X' % id(adder))
    return adder

add39 = make_adder(39)
add39(109)

id(x): 100288E40
id(adder): 104823620


148

In [2]:
# let's use repr so we can see the address of the function
# we could use print("%X") as well...
type(add39), repr(add39)

(function, '<function make_adder.<locals>.adder at 0x104823620>')

In [3]:
# all functions have a closure attribute
add39.__closure__

(<cell at 0x1047fe738: int object at 0x100288e40>,)

In [4]:
# notice that the cell object has a reference to an int object
add39.__closure__[0].cell_contents

39

* One case where closures are frequently used is in building function wrappers
* Suppose we want to log each invocation of a function:

In [8]:
def logging(f):
    def wrapper(*args, **kwargs):
        print('Calling %r(%r, %r)' % (f, args, kwargs))
        return f(*args, **kwargs)
    return wrapper

logging_add39 = logging(add39)
print(add39(5)) # remember that add39 just adds 39 to our argument
logging_add39(10)

44
Calling <function make_adder.<locals>.adder at 0x104823620>((10,), {})


49

In [9]:
logging_add39.__closure__[0].cell_contents

<function __main__.make_adder.<locals>.adder>

## Decorators
* Wrapper functions are so common, that Python has its own term for it–a *decorator*.
* Why might you want to use a decorator?
  * sometimes you want to modify a function’s behavior without explicitly modifying the function, e.g., pre/post actions, debugging, etc. 
  * suppose we have a set of tasks that need to be performed by many different functions, e.g.,
   * access control
   * cleanup
   * error handling
   * logging
 * ...in other words, there is some boilerplate code that needs to be executed before or after  every invocation of the function


## Decorators build on topics we already know...
* nested functions
* variable positional args (`*args`)
* variable keyword args (`**kwargs`)
* functions are objects (actually everything in Python is an object)

In [10]:
def document_it(func):
    # below is a nested, or inner function
    def new_function(*args, **kwargs):
        print('Running function: {}'
              .format(func.__name__))
        print('Positional arguments: {}'
              .format(args))
        print('Keyword arguments: {}'
              .format(kwargs))
        # here we invoke the function passed in as an argument
        result = func(*args, **kwargs)
        print('Result: {}'.format(result))
        return result
    
    # document_it() is returning a reference to the inner function
    return new_function

def add_ints(a, b):
    return a + b

print('Running plain old add_ints()')
print(add_ints(13, 5))

# manual decorator assignment
cooler_add_ints = document_it(add_ints) 

print('Running cooler add_ints()')
cooler_add_ints(13, 5)

Running plain old add_ints()
18
Running cooler add_ints()
Running function: add_ints
Positional arguments: (13, 5)
Keyword arguments: {}
Result: 18


18

In [13]:
# decorator shorthand for what we did above
@document_it # add_ints = document_it(add_ints)
def add_ints(a, b):
    return a + b

add_ints(4, 7)

Running function: add_ints
Positional arguments: (4, 7)
Keyword arguments: {}
Result: 11


11

## Lab: Decorators
1. Create a function called __`printer`__ that takes a string and prints it
  * Then create a wrapper that will print the number of times each letter appears in the string passed in to __`printer`__, followed by the string.
  * Use the wrapper as a decorator on your __`printer`__ function.

2. Create some function which takes an integer as its parameter
  * Create a wrapper that ensures the parameter is positive
  * use that wrapper to decorate your original function

In [14]:
from coord2d import Coord2d
c1 = Coord2d(1, 2)
c2 = Coord2d(-2, -3)

In [15]:
c1, c2

(Coord2d(x=1, y=2), Coord2d(x=-2, y=-3))

In [16]:
print(c1, c2)

Coord2d(x=1, y=2) Coord2d(x=-2, y=-3)


In [17]:
c1.distance(c2)

5.830951894845301

In [18]:
c1.add(c2)

type(a)= <class 'coord2d.Coord2d'>
type(b)= <class 'coord2d.Coord2d'>


Coord2d(x=1, y=2)

In [19]:
c2.sub(c1)

type(a)= <class 'coord2d.Coord2d'>
type(b)= <class 'coord2d.Coord2d'>


Coord2d(x=0, y=0)